In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [4]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [5]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [6]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [7]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [8]:
fig, axes = plt.subplots(2)
axes[0].plot(seq.input)
axes[1].plot(seq.target)
plt.show()

In [9]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [10]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [11]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [12]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [13]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [14]:
sample = source.get_batch(num_seq_per_batch=1024)

In [15]:
pipeline = DataPipeline(
    sources=[source, ras],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [16]:
report = pipeline.report()
report

{'pipeline': {'input_processing': [{'divisor': 980.7771, 'name': 'DivideBy'}],
  'num_seq_per_batch': 64,
  'rng_seed': None,
  'source_probabilities': [0.5, 0.5],
  'sources': {0: {'allow_incomplete_distractors': True,
    'allow_incomplete_target': False,
    'distractor_inclusion_prob': 1.0,
    'include_incomplete_target_in_output': True,
    'name': 'SyntheticAggregateSource',
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 1.0,
    'uniform_prob_of_selecting_each_building': True},
   1: {'allow_incomplete_target': True,
    'allow_multiple_target_activations_in_aggregate': False,
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'include_incomplete_target_in_output': True,
    'include_multiple_targets_in_output': False,
    'name': 'RealAggregateSource',
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 0.5,

In [17]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [18]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [19]:
from pymongo import MongoClient
client = MongoClient()
db = client.neuralnilm

In [20]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["0"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

Database already has an experiment with _id == 0. Should the old experiment be deleted (both from the database and from disk)? Or quit? [Q/d] d


INFO:neuralnilm.trainer:Deleting documents for old experiment.
INFO:neuralnilm.trainer:Directory exists = '~/temp/neural_nilm/output/0'
INFO:neuralnilm.trainer:  Deleting directory.


In [21]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo
type(sanitise_dict_for_mongo(report)['data']['pipeline']['input_processing'][0]['divisor'])

float

In [ ]:
sanitise_dict_for_mongo(report)

{'_id': '0',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 980.777099609375,
     'name': 'DivideBy'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.5, 0.5],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 1.0,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'rng_seed': None,
     'sample_period': 6,
     'seq_length': 256,
     'target_appliance': 'kettle',
     'target_inclusion_prob': 1.0,
     'uniform_prob_of_s

In [ ]:
trainer.fit(50000)

INFO:neuralnilm.trainer:Starting training for 50000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   1.221413  |    2.462075     |   0

INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.



          1 |   0.626569  |    0.007162     |   1
          2 |   0.497573  |    0.008382     |   1
          3 |   0.477277  |    0.003106     |   1
          4 |   0.619296  |    0.004985     |   1
          5 |   0.500404  |    0.196349     |   1
          6 |   1.096536  |    0.073188     |   0
          7 |   1.092271  |    0.133012     |   0
          8 |   1.039312  |    0.106207     |   0
          9 |   0.490150  |    0.159201     |   1
         10 |   0.566782  |    0.199043     |   1
         11 |   0.515970  |    0.159629     |   1
         12 |   0.530176  |    0.196903     |   1
         13 |   1.052467  |    0.137783     |   0
         14 |   1.047765  |    0.086376     |   0
         15 |   0.429886  |    0.194336     |   1
         16 |   0.669404  |    0.190181     |   1
         17 |   0.560497  |    0.203656     |   1
         18 |   1.140693  |    0.093700     |   0
         19 |   0.616026  |    0.183304     |   1
         20 |   1.007358  |    0.131912     |   0

In [ ]:
net.train_iterations

In [ ]:
from neuralnilm.monitor.monitor import Monitor

mon = Monitor("0")

In [ ]:
mon._plot_train_scores()

In [ ]:
mon._plot_validation_scores()

In [ ]:
mon.validation_metric_names

In [ ]:
mon.db.validation_scores.distinct(key='source_id')

In [ ]:
bool(trainer.db.train_scores.find_one({'experiment_id': trainer.experiment_id}))

In [ ]:
trainer.db.train_scores.delete_many?